# Basic signal processing with ${\tt python}$

Every python function is documented on the internet. google the name of the function will provide you with the link to its online description. For example, see
[http://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html](http://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html).


## Part 1 -  A synthetic case
1. The example below shows how to generate a synthetic timeseries between 1850.0 AD and 2010.0 AD using the ${\tt synthetic.py}$ module, that contains a single harmonic component of period 10 yr, and is evenly sampled every 12th of a year (ie every month). 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from synthetic import *

t_start = 1850.0
t_end = 2010.0
t_samp = 1./12.

# define synthetic time series
mytimewindow = def_timewindow(t_start, t_end, t_samp)
period = 10. # in yr
amp = 1.
phase = -2*np.pi*t_start/period
mytseries = cos_tseries(mytimewindow, period, amp, phase)
# plot it
fig = plt.figure( figsize = (10, 4) ) #figsize can be adjusted to your needs
ax = fig.add_subplot(111)
ax.plot(mytimewindow, mytseries)
ax.set_xlabel('year AD')
ax.set_ylabel('signal')
ax.set_xlim(t_start,t_end)
ax.set_xticks(np.linspace(t_start, t_end, endpoint = True, num = int( (t_end - t_start)/10 + 1 ) ) ) 
# show the figure
plt.show()


2. Your turn now: Using the python functions at your disposal in the ${\tt synthetic.py}$ module, create a synthetic timeseries spanning the [1850.0~AD-2010.0~AD] time window (with one point every month), comprising four harmonic components, of periods $(T_1,T_2,T_3,T_4)=(0.5,1.0,11.0,60.0)$~yr, respectively. You can arbitrarily set the amplitude $A_i$ and the phase $\varphi_i$ of each of these components. The signal $s(t)$ so created reads $$
        s(t) = \sum_{i=1}^{i=4} s_i(t) = \sum_{i=1}^{i=4} A_i \cos \left( 2 \pi t / T_i +\varphi_i \right). 
        $$

In [ ]:
# write your own code below
#
# Define your timeseries. I would suggest to use a name different from mytseries in the example above, to avoid a source of 
# confusion between the example and your own code
#



#Plot it


#

$$\newcommand{\fourier}[1]{\widehat{#1}}$$
2. We wish to perform the spectral analysis of $s(t)$ and therefore to compute its (discrete) Fourier transform. Let $\fourier{s}(f)$ denote this transform, with $f$ the frequency (inverse of the period) - keeping in mind that the angular frequency $\omega=2\pi f$.To compute $\fourier{s}$, we resort to python again, in particular to the ${\tt fft.rfft}$ function. An example of its usage is given in the piece of code below, where the spectral analysis is performed on the harmonic example I gave above

In [ ]:
# compute its Fourier transform
myshat = np.fft.rfft(mytseries) # 
# these are the associated frequencies (useful for subsequent plot)
myfreq = np.fft.rfftfreq(len(mytseries),t_samp) # in year^{-1} 
# plot the timeseries and its power spectrum 
fig = plt.figure( figsize=(10,7) )
#
ax1 = fig.add_subplot(211)
ax1.plot(mytimewindow, mytseries)
ax1.set_xlabel('year AD')
ax1.set_ylabel('signal')
ax1.set_xlim(t_start, t_end)
ax1.set_xticks(np.linspace(t_start, t_end, endpoint = True, num = int((t_end - t_start)/10 + 1 )))
#
ax2 = fig.add_subplot(212)
ax2.loglog(myfreq, abs(myshat)**2)
ax2.set_xlabel('frequency in yr$^{-1}$')
ax2.set_ylabel(r'|$\hat{s}$|$^2$')
# show the figure
plt.show()


Your turn now: Adapt this example to represent the squared modulus of $\fourier{s}(f)$

In [ ]:
#write your own code below
#

3. Now let us get acquainted with the Butterworth filters provided by the ${\tt signal}$ processing package of scientific python. The example below shows how to apply a bandpass filter, with corner periods equal to 5 and 15 yr, to the example timeseries I have been using so far. 

In [ ]:
from scipy import signal 
# define Butterworth filter properties
#
# The frequency band [w_low_cut,w_high_cut] is given as a fraction of the 
# Nyquist Frequency which is equal to half of the sampling frequency
# 
T_long  = 15 # in yr
T_short = 5  # in yr
#
f_samp = 1./t_samp
f_nyq =  f_samp / 2
#
f_low_cut =  (1./T_long)  # in year^{-1}
w_low_cut =  f_low_cut / f_nyq # angular frequency
#
f_high_cut = (1./T_short)  # in year^{-1}
w_high_cut = f_high_cut / f_nyq          # angular frequency
# define filter properties
b,a = signal.butter(4, [w_low_cut,w_high_cut], 'band')
# apply filter to signal (twice, to avoid phasing) 
recons = signal.filtfilt(b,a,mytseries)
# compute its Fourier transform
myreconshat = np.fft.rfft(recons)
#
# plot the timeseries and its power spectrum, as well as those
# of the filtered signal 
fig = plt.figure( figsize = (10, 7) )
#
ax1 = fig.add_subplot(211)
ax1.plot(mytimewindow, mytseries)
ax1.plot(mytimewindow, recons)
ax1.set_xlabel('year AD')
ax1.set_ylabel('signal')
ax1.set_xlim(t_start,t_end)
ax1.set_xticks(np.linspace(t_start, t_end, endpoint = True, num = int( (t_end - t_start)/10 + 1 )) )
#
ax2 = fig.add_subplot(212)
ax2.loglog(myfreq, abs(myshat)**2, label="original")
ax2.loglog(myfreq, abs(myreconshat)**2, label="filtered")
ax2.set_xlabel('frequency in yr$^{-1}$')
ax2.set_ylabel(r'|$\hat{s}$|$^2$')
ax2.legend()
# show the figure
plt.show()

Your turn now: extract in $s(t)$ the $11$-yr component. Plot the filtered signal against the original one, and compare as well its spectrum to the original one.

In [ ]:
# Write your code below
#
#

4. By looking at the manual page of ${\tt scipy.signal.filtfilt}$, the default version of which we use, suggest a way of improving the reconstruction of the sought harmonic component. This improvement can follow from the artificial augmentation of the length of the timeseries (padding it with zeroes), which tends to mitigate spurious edge effects on the filtered signal.

In [ ]:
# Write your answer here (code not necessarily needed)
#

## Part 2 - A real case: 78 years of geomagnetic measurements @ CLF

CLF : Chambon-la-Forêt, where the French magnetic observatory is located since 1936. See e.g. [www.bcmt.fr](http://www.bcmt.fr). 

The file ${\tt clf1936-2014.dat}$ contains the monthly means of the geomagnetic elements recorded at the CLF observatory. 
The header of the file looks like so
~~~~
# This file is provided by the database of the "Bureau Central de   |
# Magnetisme Terrestre" (BCMT, France).                             |
# Conditions of use: these data are for scientific/academic use.    |
# Formula for computing non-reported elements:                      |
# X=H*cos(D), Y=H*sin(D), tan(I)=Z/H                                |
# D is expressed in minutes of arc.                                 |
# 1-month values are computed from 24 monthly means of 1-hour       |
# values.                                                           |
# For any enquiry, please contact: bcmt@ipgp.fr                     |
#DATE       TIME        DOY     CLFH      CLFD      CLFZ      CLFF  
~~~~

1. The code below reads the file and and plots the fluctuations of the vertical component ($Z$) at the site between 1936.0 and 2015.0. 

In [ ]:
import matplotlib.dates as dts
import time

datestr2num = lambda s: dts.datestr2num(s.decode('ascii'))
#
fname = 'clf1936-2014.dat'
#
#
data = np.loadtxt(fname, usecols=(0,3,4,5,6), converters={0:datestr2num})
#
epoch = data[:,0]
decl = data[:,2]/60. # declination expressed in degrees
zcomp = data[:,3]
#
years = dts.YearLocator()   # every year
months = dts.MonthLocator()  # every month
yearsFmt = dts.DateFormatter('%Y') # format for a year
#
# Plot timeseries of Z
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.plot(epoch,zcomp,'k+')
# format the ticks
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
datemin = dts.datestr2num('1936-01-01')
datemax = dts.datestr2num('2016-01-01')
ax.set_xlim(datemin, datemax)
ax.set_xticks(np.arange(datemin,datemax,1826))
ax.set_xlabel('year AD')
ax.set_ylabel('$Z$ (nT)')
fig.tight_layout()
plt.show()

Write your own code to compute and plot $X(t)$ @ CLF, $X$ being the Northward component, whose connection with the horizontal component and the declination is given in the header above. 

In [ ]:
# your code below
#

2. Based on the expertise you gained in the first synthetic part of this lab, write a code that, in addition to plotting $X(t)$, computes $\fourier{X}(f)$ and plots the power spectral density of $X$. It is recommended to have $X(t)$ and $|\fourier{X}|^2(f)$ on the same plot (with two graphs). 

In [ ]:
#your code below
#

3. Analyze the spectrum so obtained. Are there any obvious peaks? Which periods do they correspond to? Can you make sense of them? 

In [ ]:
Your answer: 

4. Write another piece of code to filter out the long-period component of $X(t)$, with periods greater than 15~years, say. Plot the original $X(t)$ and its filtered version $X'(t)$ on one graph, and the spectra of $|\fourier{X}|^2(f)$ and $|\fourier{X'}|^2(f)$ on a second graph (both graphs on the same plot). 

In [ ]:
#your code below
#

5. The file ${\tt SN.dat}$ contains monthly values of the sunspot number (a proxy for solar activity) between 1749 and 2016. Read it and plot it against $X'$ (one graph per signal, two graphs on the same plot), for that period of time during which there is data for both (1936--2015). 

In [ ]:
#your code below
#

6. Is there a connection between the two? If yes, propose an interpretation. 

**Bonus**: compute and plot the power spectrum density based on the entire sunspot number 
      timeseries.

In [ ]:
#your code below